# Lecture 4: Password CPA Attack - Attack

In this example we want to improve the password check again to be resistant against the attack from the last tutorial.

## Improving the code

Let's first recap the password checking loop from the last lecture:
```c
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    if (stored_password[i] != passwd[i])
    {
        password_wrong = 1;
    }
}
```

The differences attack discussed in the last example worked because of the different power consumption when executing the code inside the if clause. This is addressed by the following code.

```c
uint8_t password_wrong = 0;
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    password_wrong |= stored_password[i] ^ passwd[i];
}
```

This is an excerpt from `4_password_fixed.c`.

## Pearson correlation coefficient
An interesting statistical formula to face this problem is given by the *Pearson correlation coefficient*. For two random variables $X, Y$ it is defined as

$$\rho_{X,Y} := \frac{\mathrm{Cov}(X, Y)}{\sqrt{\mathrm{Var}(X)} \sqrt{\mathrm{Var}(Y)}} \ \in [-1, 1]\,.$$

For two samples of finite length $x = {x_1, ..., x_n}$, $y = {y_1, ..., y_n}$ it can be defined as 

$$r_{x,y} := \frac{\sum_{i=1}^n (x_i - \bar x)(y_i - \bar y)}{\sqrt{\sum_{i=1}^n (x_i - \bar x)^2}\sqrt{\sum_{i=1}^n (y_i - \bar y)^2}} \ \in [-1, 1]\,,$$

where $\bar x := \frac{1}{n} \sum_{i=1}^n x_i$ is the mean of a sample $x$.

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 1</div>
    
1. Implement Pearson correlation coefficient in python.
2. Test your implementation against the following (x, y)-data with `size = 50`:
    1. `(range(size), 5 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size))` => $r \approx 0.99$
    2. `(range(size), np.array(range(size)) + np.random.uniform(-size, size, size=size) + 10)` => $r \approx 0.32$
    3. `(range(size), -3 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size) + 200)` => $r \approx -0.98$
    4. `(range(size), 100 * np.sin(np.array(range(size)) / size * np.pi + np.pi) + np.random.uniform(-size/10, size/10, size=size))` => $r \approx -0.05$
3. Visualize above example data.
</div>

## Attack

How to use this as principle for an attack?

We sae in the previous notebook that there is a _linear_ relationship between the trace at a given point and the hamming weight of the attempt. Now we can quantify a linear dependency!

So, we can develop an attack out of this principle:

1. Record traces with random input.
2. Focus on a point in the program where the input "collides" with the secret. In our case this is the XOR.
3. For each input compute a "forecast" of hamming weights. If we do this for all possible secrets we will see that there is one guess where the Pearson coefficient between the forecast and the traces at this certain point maximizes.

It is still open how to find the sweet spot. We just don't try to "find" it. We're just computing the Pearson coefficient for _every_ trace point and keep the best one.

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 2</div>
    
Develop an attack using Pearson correlation coefficient.

1. Start with the first character:
   1. Capture a few hundred (~500) traces with random input.
   2. For each guess: Calculate the pointwise Pearson correlation between the forecast and the traces. Keep the absolute maximum of the correlations.
   3. Now keep the guess with the maximal correlation.
2. Extend the attack to all other characters.

Note: Due to interferences the correct character might not be the one with the highest Pearson coefficient. Thus it is worth to keep the best 2-3 guesses.

</div>